# Deep learning. Practical assignment 1. Multilayer perceptron on numpy.

In [ ]:
import numpy as np

In this task, you will implement multilayer perceptron (MLP) that recognizes handwritten digits. Your task is to implement forward and backward pass through a fully connected layer and layers with nonlinearity and then assemble a model from these layers. Also, you are to implement an advanced linear layer (with orthogonal weight matrix) and to compare this model with the ordinary one.

[__ Notes with formulas__](https://github.com/nadiinchi/dl_labs/blob/master/nn_gradients.pdf)

All our layers will follow the same interface:

In [ ]:
class IdentityLayer:
    """
    A building block. Each layer is capable of performing two things:
    
    - Process input to get output:           
    output = layer.forward(input)
    
    - Propagate gradients through itself:    
    grad_input = layer.backward(input, grad_output)
    
    Some layers also have learnable parameters.
    """
    def __init__(self):
        """Here you can initialize layer parameters (if any) 
        and auxiliary stuff. You should enumerate all parameters
        in self.params"""
        # An identity layer does nothing
        self.params = []
        pass
    
    def forward(self, input):
        """
        Takes input data of shape [batch, input_units], 
        returns output data [batch, output_units]
        """
        # An identity layer just returns whatever it gets as input.
        self.input = input
        return input

    def backward(self, grad_output): 
        """
        Performs a backpropagation step through the layer, 
        with respect to the given input.
        
        To compute loss gradients w.r.t input, 
        you need to apply chain rule (backprop):
        
        d loss / d input  = (d loss / d layer) *  (d layer / d input)
        
        Luckily, you already receive d loss / d layer as input, 
        so you only need to multiply it by d layer / d x.
        
        The method returns:
        * gradient w.r.t input (will be passed to 
          previous layer's backward method)
        * flattened gradient w.r.t. parameters (with .ravel() 
          applied to each gradient). 
          If there are no params, return []
        """
        # The gradient of an identity layer is precisely grad_output
        input_dim = self.input.shape[1]
        
        d_layer_d_input = np.eye(input_dim)
        
        return np.dot(grad_output, d_layer_d_input), [] # chain rule

### Implementation of ReLU and linear layers

Let's begin with a simple nonlinearity layer $ReLU(x) = max(x, 0)$. This layer has no parameters. Method .forward should return the result of element-wise applying ReLU to the input array. Method .backward should return gradient w. r. t. the input of the layer. We will assume that the derivative of ReLU in 0 equals 0. Please note that on the backward pass you may need some values computed during the forward pass so you should save them as an attribut of the class.

In [ ]:
class ReLU:
    def __init__(self):
        """ReLU layer simply applies elementwise rectified linear unit to all inputs"""
        self.params = [] # ReLU has no parameters
    
    def forward(self, input):
        """Apply elementwise ReLU to [batch, num_units] matrix"""
        ### your code here
        
    
    def backward(self, grad_output):
        """Compute gradient of loss w.r.t. ReLU input
        grad_output shape: [batch, num_units]
        output shape: [batch, num_units]
        """
        ### your code here
        

Now implement a fully-connected layer without nonlinearity. This layer has two parameters: weight matrix and bias vector.

In [ ]:
class Dense:
    def __init__(self, input_units, output_units):
        """
        A dense layer is a layer which performs a learned affine transformation:
        f(x) = W x + b
        """
        # initialize weights with small random numbers from normal distribution
        self.weights = np.random.randn(output_units, input_units)*0.01
        self.biases = np.zeros(output_units)
        self.params = [self.weights, self.biases]
        
    def forward(self,input):
        """
        Perform an affine transformation:
        f(x) = W x + b
        
        input shape: [batch, input_units]
        output shape: [batch, output units]
        """
        ### your code here
        
    
    def backward(self, grad_output):
        """
        compute gradients
        grad_output shape: [batch, output_units]
        output shapes: [batch, input_units], [num_params]
        
        hint: use function np.r_
        np.r_[np.arange(3), np.arange(3)] = [0, 1, 2, 0, 1, 2]
        """
        ### your code here
        

### Gradient checking

Let's check whether the gradient computation is correct. The following function takes callable object as input (function of 1 array) and a point and computes the approximate gradient in this point.

In [ ]:
def eval_numerical_gradient(f, x, verbose=False, h=0.00001):
    """Evaluates gradient df/dx via finite differences:
    df/dx ~ (f(x+h) - f(x-h)) / 2h
    Adopted from https://github.com/ddtm/dl-course/
    """
    fx = f(x) # evaluate function value at original point
    grad = np.zeros_like(x)
    # iterate over all indexes in x
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:

        # evaluate function at x+h
        ix = it.multi_index
        oldval = x[ix]
        x[ix] = oldval + h # increment by h
        fxph = f(x) # evalute f(x + h)
        x[ix] = oldval - h
        fxmh = f(x) # evaluate f(x - h)
        x[ix] = oldval # restore

        # compute the partial derivative with centered formula
        grad[ix] = (fxph - fxmh) / (2 * h) # the slope
        if verbose:
            print (ix, grad[ix])
        it.iternext() # step to next dimension

    return grad

Firstly, compute analytical and numerical gradient of the following function with respect to the input of ReLU:
$$ f(y) = \sum_i y_i, \quad y = ReLU(x) $$

Your code should use .forward and .backward methods of ReLU class.
The next cell should print "OK":

In [ ]:
x = np.linspace(-1, 1, 10*12).reshape([10, 12])
### your code here
### grads = 
### numeric_grads = 


assert np.allclose(grads, numeric_grads, rtol=1e-3, atol=0)
print("OK")

Now compute analytical and numerical gradient with respect to the __input__ of the linear layer:
$$ f(y) = \sum_i y_i, \quad y = W x + b $$

In [ ]:
x = np.linspace(-1, 1, 10*12).reshape([10, 12])
l = Dense(12, 32)
### your code here
### grads = 
### numeric_grads = 


assert np.allclose(grads, numeric_grads, rtol=1e-3, atol=0)
print("OK")

Now compute analytical and numerical gradient with respect to the __parameters__ of the linear layer:

In [ ]:
x = np.linspace(-1, 1, 10*12).reshape([10, 12])
weights = np.linspace(-1, 1, 12*7).reshape([7, 12]) / 100
biases = np.linspace(-1, 1, 7)
vec = np.r_[weights.ravel(), biases]
### your code here
### grads = 
### numeric_grads = 


assert np.allclose(grads, numeric_grads, rtol=1e-3, atol=0)
print("OK")

### Implementation of softmax layer and loss function

In classification tasks, the last layer of the network is usually a softmax layer that outputs probabilities of each class for an object:
$$\hat y = softmax(x)  = \biggl \{\frac {exp(x_k)}{\sum_j exp(x_j)} \biggr \}_{k=1}^K, \quad K - \text{number of classes}$$
In this case, it is convinient to optimize the likelihood
$$L(y, \hat y) = -\sum_{k=1}^K y_k \log \hat y_k \rightarrow \min,$$
where $y_k=1$ if an object belongs to the $k$-th class and 0 otherwise. Written in a such form, this funcion matches the formula of cross-entropy. Obviously, we can rewrite the loss using indexing and denoting a class of the object with $y$:
$$L(y, \hat y) = - \log \hat y_{y} \rightarrow \min$$

Implement Softmax layer (no parameters). Method .forward should return the __logarithm__ of softmax, and method .backward should propagate the gradients. We will assume that the argument grad_output is a matrix in which each row contains only one non-zero element (not necessarily 1, may be $\frac 1 N$ etc.).

In [ ]:
from scipy.misc import logsumexp
# use this function instead of np.log(np.sum(np.exp(...))) !
# because it is more stable

In [ ]:
class Softmax:
    def __init__(self):
        self.params = []
        
    def forward(self, input):
        """
        Applies softmax to each row and then applies component-wise log
        Input shape: [batch, num_units]
        Output shape: [batch, num_units]
        """
        ### your code here
        
    
    def backward(self, grad_output):
        """
        Propagartes gradients.
        Assumes that each row of grad_output contains only 1 
        non-zero element
        Input shape: [batch, num_units]
        Output shape: [batch, num_units]
        Do not forget to return [] as second value (grad w.r.t. params)
        """
        ### your code here
        

Implement the loss function and its gradients:

In [ ]:
def crossentropy(activations, target):
    """
    returns negative log-likelihood of target under model represented by
    activations (log probabilities of classes)
    activations shape: [batch, num_classes]
    target shape: [batch] (array of integers)
    output shape: 1 (scalar)
    """
    ### your code here
    

def grad_crossentropy(activations, target):
    """
    returns gradient of negative log-likelihood w.r.t. activations
    activations shape: [batch, num_classes]
    target shape: [batch] (array of integers)
    output shape: [batch, num-classes]
    
    hint: this is just one-hot encoding of target vector
          multiplied by -1
    """
    ### your code here
    

Finally, check the gradient of the softmax-layer, using the loss function and its gradient:

In [ ]:
points = np.linspace(-1, 1, 10*12).reshape([10, 12])
target = np.arange(10)
### your code here
### grads = 
### numeric_grads = 


assert np.allclose(grads, numeric_grads, rtol=1e-3, atol=0)
print("OK")

### Data loading
We have implemented all the basic layers of our neural network. Now let's load the data and train our model. We will work with digits dataset in which each objects represents 8x8 grayscale image of a handwritten digit:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.datasets import load_digits

In [ ]:
X, y = load_digits(return_X_y=True)

In [ ]:
X.shape, y.shape

Split the data into train and test:

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
X_train.shape, X_test.shape

### Assembling and training the model

In out implementation, the neural network is a list of layers. For example:

In [ ]:
network = []
hidden_layers_size = 32
network.append(Dense(X_train.shape[1], hidden_layers_size))
network.append(ReLU())
network.append(Dense(hidden_layers_size, hidden_layers_size))
network.append(ReLU())
network.append(Dense(hidden_layers_size, 10))
network.append(Softmax())

To check the quality of our neural network, we will compute accuracy. To do this, implement the function that makes predictons using the model:

In [ ]:
def predict(network, X):
    """
    returns predictions for each object in X
    network: list of layers
    X: raw data
    X shape: [batch, features_num]
    output: array of classes, each from 0 to 9
    output shape: [batch]
    """
    ### your code here
    

We will learn the parameters of the network using scipy.optimize:

In [ ]:
from scipy.optimize import minimize

In [ ]:
help(minimize)

This function has a standard interface: you have to pass a callable object  (computes the value of the loss and the gradients) and a point where to start the optimization (1-d np.ndarray). That's why we will need functions that set and collect all the parameters of our network (for this purpose, we always saved parameters of the layer in .params attribute):

In [ ]:
def get_weights(network):
    weights = []
    for layer in network:
        for param in layer.params:
            weights += param.ravel().tolist()
    return np.array(weights)

def set_weights(weights, network):
    i = 0
    for layer in network:
        for param in layer.params:
            l = param.size
            param[:] = weights[i:i+l].\
                             reshape(param.shape)
            i += l
        

You have to implement the function that we will pass to the minimize function. Function compute_loss_grad takes current point as input (vector containing all the parameters of the network) and a list of additional parameters (we will pass the net and training data through it). The function outputs the value of the loss and its gradients w. r. t. parameters of the model.

In [ ]:
def compute_loss_grad(weights, args):
    """
    takes current weights and computes cross-entropy and gradients
    weights shape: [num_parameters]
    output 1: loss (scalar)
    output 2: gradint w.r.t. weights, shape: [num_parameters]
    
    hint: firstly perform forward pass through the whole network
    then compute loss and its gradients
    then perform backward pass, transmitting first baskward output
    to the previos layer and saving second baskward output in a list
    finally flatten all the gradients in this list
    (in the order from the first to the last layer)
    
    Do not forget to set weights of the network!
    """
    network, X, y = args
    ### your code here
    

Now we are ready to train our network:

In [ ]:
weights = get_weights(network)

In [ ]:
res = minimize(compute_loss_grad, weights,  # fun and start point
               args=[network, X_train, y_train], # args passed to fun
               method="L-BFGS-B", # optimization method
               jac=True) # says that gradient are computed in fun

In [ ]:
res.keys()

In [ ]:
res["nit"] # number of iterations (should be >> 10)

In [ ]:
res["success"] # should be True

In [ ]:
res["x"] # leraned weights

Compute the accuracy of the network on the training set (X_train, y_train) and on the testing set (X_test, y_test). Do not forget to set the weights of the network!

In [ ]:
### your code here


Function minimize also has additional callable argument callback. This function callback will be called after each iteration of the optimization. It is convinient to implement this function as a method of a class. This method will save the train and the test quality after each epoch. Implement class Callback:

In [ ]:
class Callback:
    def __init__(self, network, X_train, y_train, X_test, y_test, print=False):
        self.network = network
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.print = print
        self.train_acc = []
        self.test_acc = []
        
    def call(self, weights):
        """
        computes quality on train and test set with given weights
        and saves to self.train_acc and self.test_acc
        if self.print is True, also prints these 2 values
        """
        ### your code here
        

In [ ]:
cb = Callback(network, X_train, y_train, X_test, y_test, print=True)
res = minimize(compute_loss_grad, weights,  
               args=[network, X_train, y_train], 
               method="L-BFGS-B",
               jac=True,
               callback=cb.call)

Plot the training progress:

In [ ]:
plt.plot(cb.train_acc, label="train acc")
plt.plot(cb.test_acc, label="test acc")
plt.xlabel("Iteration")
plt.ylabel("Accuracy")
plt.legend()

### Experiment with the number of the layers

Fill in the tables accs_train and accs_test. In position [i, j] each table contains the accuracy of the network with $i+1$ layers on the $j$-th restart (all restarts are identical). Restarts are needed because the quality heavily depends on the random initialization.

In [ ]:
accs_train = np.zeros((10, 5))
accs_test = np.zeros((10, 5))

In [ ]:
num_hidden_size = 10 # for all layers
### your code here


Let's plot the boxplots of the quality:

In [ ]:
plt.boxplot(accs_train.T, showfliers=False)
plt.xlabel("Num layers")
plt.ylabel("Train accuracy")
plt.title("Train quality in 5 runs")

In [ ]:
plt.boxplot(accs_test.T, showfliers=False)
plt.xlabel("Num layers")
plt.ylabel("Test accuracy")
plt.title("Test quality in 5 runs")

As you can see, the quality drops when the number of layer become big. That is because of the vanishing gradients problem. One of the approaches to overcome this problem is to use layers with orthogonal matrices. Let's disscuss and implement this approach.

### Network with orthogonal matrices

In our case, vanishing gradients problem comprises that the norm of the gradients with respect to the parameters of the first layers is small. One of possible solutions is to use fully-connected layers with orthogonal matrices. These matrices have useful norm-preserving property: if $y = Wx$ and $W^T W = I$, then
$$ \| y \|^2 =  \| Wx \|^2  = x^T W^T W x = x^T x =  \| x \|^2. $$
As a result, the distibution of the preactivations is preserved during forward pass and the norm of the gradients is preserved during backward pass. Of course, it doesn't solve the problem completely because of the nonlinearities.

Let's implement the fully-connected layer where weight matrix $W$ is always orthogonal. It can be achieved by using the following parametrization of the layer:
$$ W = U (U^T U)^{- \frac 1 2} $$
$$ y = W x + b. $$
Here $U$ and $b$ are the parameters of the layer (with the only constraint that $\| U \| \leq 1$ that can be achieved by scaling of the matrix).

To compute $(U^T U)^{- \frac 1 2} = A^{- \frac 1 2}$, one can use Newton Schulz iterations:
$$Y_0 = A, \,Z_0 = I$$
$$Y_{k+1} = \frac 1 2 Y_k (3 I - Z_k Y_k), \, Z_{k+1} = \frac 1 2 (3 I - Z_k Y_k) Z_k.$$
Then $Y_k \rightarrow A^{\frac 1 2}, Z_k \rightarrow A^{-\frac 1 2}$. For the details and other approaches, please refer to [these notes](https://people.cs.umass.edu/~smaji/projects/matrix-sqrt/).

Your last task is to implement described layer. On the forward pass, perform Newton Schulz iterations to compute $W$ and then apply usual linear transformation. On the backward pass, backpropagate gradients firstly through linear transformation (as in Dense layer) and then through Newton Schulz iterations. Our layer will have square weight matrix $U$.

In [ ]:
class DenseOrthogonal:
    def __init__(self, num_units, num_steps=20):
        """
        This layer which performs a learned affine transformation 
        with orthogonal matrix parametrixation:
        W = U (U^T U) ^ {-1/2}
        f(x) = W x + b
        """
        m = num_units
        self.U = np.random.randn(m, num_units)*0.001
        self.biases = np.zeros(m)
        self.params = [self.U, self.biases]
        self.num_steps = num_steps
        self.I = np.eye(num_units)
        
    def forward(self, input):
        """
        Perform an affine transformation:
        W = U (U^T U) ^ {-1/2}
        f(x) = W x + b
        
        input shape: [batch, input_units]
        output shape: [batch, output units]
        """
        ### your code here
    
    def backward(self, grad_output):
        """
        compute gradients
        grad_output shape: [batch, output_units]
        output shapes: [batch, input_units], [num_params]
        
        """
        ### your code here

When you implement the layer, check the gradients w. r. t input of the layer and w. r. t. parameters of the layer as you did with Dense layer:

In [ ]:
### your code here



After that, repeat the experiment with different number of layers but using Dense Orthogonal layer instead of Dense.

In [ ]:
accs_train_orth = np.zeros((10, 5))
accs_test_orth = np.zeros((10, 5))

In [ ]:
num_hidden_size = 10 # for all layers
### your code here


In [ ]:
plt.boxplot(accs_train_orth.T, showfliers=False)
plt.xlabel("Num layers")
plt.ylabel("Train accuracy")
plt.title("Train quality in 5 runs")

In [ ]:
plt.boxplot(accs_test_orth.T, showfliers=False)
plt.xlabel("Num layers")
plt.ylabel("Test accuracy")
plt.title("Test quality in 5 runs")